In [1]:
!python -m pip install --upgrade pip

Requirement already up-to-date: pip in c:\users\yeonsik\anaconda3\lib\site-packages (20.0.2)


In [2]:
!pip3 install --upgrade tensorflow-gpu

Requirement already up-to-date: tensorflow-gpu in c:\users\yeonsik\anaconda3\lib\site-packages (2.1.0)


In [3]:
from tensorflow.python.client import device_lib 

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3673955663943549632
]


In [4]:
import pandas as pd

In [5]:
path = "C:/dacon/temp/"

train = pd.read_csv(path+"train.csv")
test = pd.read_csv(path+"test.csv")
sample_submission = pd.read_csv(path+"sample_submission.csv")

In [6]:
train.head()

,id,X00,X01,X02,X03,X04,X05,X06,X07,X08,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
0,0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN
3,3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,NaN
4,4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,NaN


In [42]:
Y18_null_train_x=train[train["Y18"].isnull()].iloc[:,1:40]

In [43]:
Y18_null_train_y=train[train["Y18"].isnull()].iloc[:,57]

In [62]:
print(np.array(Y18_null_train_x))

[[0.0619469  0.57788945 0.11764706 ... 0.         0.74346202 0.58127438]
 [0.04424779 0.58291457 0.16666667 ... 0.         0.74470735 0.59297789]
 [0.04867257 0.5879397  0.10784314 ... 0.         0.74470735 0.59817945]
 ...
 [0.49557522 0.52763819 0.08823529 ... 0.         0.8132005  0.74772432]
 [0.51769912 0.52763819 0.1372549  ... 0.         0.8132005  0.75292588]
 [0.52212389 0.52763819 0.11764706 ... 0.         0.82191781 0.76723017]]


In [64]:
print(np.array(Y18_null_train_y).reshape(-1,1))

[[ 8. ]
 [ 7.5]
 [ 7.5]
 ...
 [20. ]
 [20. ]
 [19.5]]


In [ ]:
# 0. 사용할 패키지 불러오기
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.utils import np_utils


from sklearn.preprocessing import MinMaxScaler

# 데이터 전처리
scaler = MinMaxScaler(feature_range=(0, 1))
Y18_null_train_x = scaler.fit_transform(Y18_null_train_x)
Y18_null_train_y = scaler.fit_transform(np.array(Y18_null_train_y).reshape(-1,1))

def create_dataset(signal_data_x, signal_data_y, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data_x)-look_back):
        dataX.append(signal_data_x[i:(i+look_back)])
        dataY.append(signal_data_y[i + look_back])
    return np.array(dataX), np.array(dataY)

Y18_null_train_x = np.array(Y18_null_train_x)
Y18_null_train_y = np.array(Y18_null_train_y)

# 랜덤시드 고정시키기
np.random.seed(5)

look_back = 7

x_train, y_train = create_dataset(Y18_null_train_x, Y18_null_train_y, look_back)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 39))

# 1. Y18 = notnull 데이터로 1차 모델 구성
model = Sequential()
model.add(LSTM(128, batch_input_shape = (1, look_back, 39), stateful=True))
model.add(Dropout(0.3))
model.add(Dense(1))

# 4. 모델 학습과정 설정하기
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# 5. 모델 학습시키기
num_epochs = 2000
for epoch_idx in range(num_epochs):
    print ('epochs : ' + str(epoch_idx) )
    model.fit(x_train, y_train, epochs=1, batch_size=1, verbose=2, shuffle=False)
    model.reset_states()

epochs : 0
Epoch 1/1
 - 5s - loss: 0.0222 - mse: 0.0222
epochs : 1
Epoch 1/1
 - 5s - loss: 0.0112 - mse: 0.0112
epochs : 2
Epoch 1/1
 - 5s - loss: 0.0076 - mse: 0.0076
epochs : 3
Epoch 1/1
 - 6s - loss: 0.0074 - mse: 0.0074
epochs : 4
Epoch 1/1
 - 5s - loss: 0.0065 - mse: 0.0065
epochs : 5
Epoch 1/1
 - 6s - loss: 0.0055 - mse: 0.0055
epochs : 6
Epoch 1/1
 - 5s - loss: 0.0048 - mse: 0.0048
epochs : 7
Epoch 1/1
 - 6s - loss: 0.0056 - mse: 0.0056
epochs : 8
Epoch 1/1
 - 6s - loss: 0.0049 - mse: 0.0049
epochs : 9
Epoch 1/1
 - 5s - loss: 0.0044 - mse: 0.0044
epochs : 10
Epoch 1/1
 - 6s - loss: 0.0052 - mse: 0.0052
epochs : 11
Epoch 1/1
 - 6s - loss: 0.0046 - mse: 0.0046
epochs : 12
Epoch 1/1
 - 5s - loss: 0.0042 - mse: 0.0042
epochs : 13
Epoch 1/1
 - 5s - loss: 0.0042 - mse: 0.0042
epochs : 14
Epoch 1/1
 - 5s - loss: 0.0042 - mse: 0.0042
epochs : 15
Epoch 1/1
 - 5s - loss: 0.0037 - mse: 0.0037
epochs : 16
Epoch 1/1
 - 5s - loss: 0.0039 - mse: 0.0039
epochs : 17
Epoch 1/1
 - 5s - loss: 0.003